# Super-structure scenario modelling in brightway

This notebook shows how to use the bw_scenarios module, using the example of a bike.

We start by importing an SDF file (.xslx or .csv).

## Project setup

In [1]:
import bw2data as bd
import bw2io as bi

In [2]:
PROJECT_NAME = "bc24-hackathon-bike-project"
BIO_DB_NAME = "ecoinvent-3.10-biosphere"
print("Check")

Check


In [3]:
# this is the current preferred way to install a biosphere database w/bw25
# it will create biosphere named "ecoinvent-3.10-biosphere"
# and add the associated LCIA methods
bi.remote.install_project(
    "ecoinvent-3.10-biosphere", PROJECT_NAME, overwrite_existing=True
)

17465344it [00:00, 29668954.24it/s]                              


Restoring project backup archive - this could take a few minutes...
Applying automatic update: 4.0 database search directories
Reindexing database biosphere
Applying automatic update: 4.0 migrations filename change
Applying automatic update: 4.0 new processed format
Updating all LCIA methods


668it [02:09,  5.16it/s]


Updating all LCI databases


3it [01:38, 32.68s/it]


Restored project: bc24-hackathon-bike-project


'bc24-hackathon-bike-project'

In [4]:
bd.projects.set_current(PROJECT_NAME)

Applying automatic update: 4.0 database search directories
Reindexing database ecoinvent-3.10-biosphere


In [5]:
bd.databases

Databases dictionary with 1 object(s):
	ecoinvent-3.10-biosphere

In [6]:
xl_importer = bi.importers.ExcelImporter("bike_production_example.xlsx")

Extracted 1 worksheets in 0.02 seconds


In [ ]:
# If there are unlinked exchanges because of biosphere name change,
# use the right biosphere name

In [7]:
for data_item in xl_importer.data:
    for exchange in data_item["exchanges"]:
        if exchange["type"] == "biosphere" and exchange["database"] == "biosphere3":
            print(f"Update bio db name for {exchange}")
            exchange["database"] = BIO_DB_NAME

Update bio db name for {'name': 'Carbon dioxide, fossil', 'amount': 26.6, 'database': 'biosphere3', 'unit': 'kilogram', 'categories': 'air', 'type': 'biosphere'}


In [8]:
xl_importer.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 16.90 seconds


In [9]:
xl_importer.statistics()

3 datasets
	8 exchanges
	Links to the following databases:
		bike_production_example (7 exchanges)
		ecoinvent-3.10-biosphere (1 exchanges)
	0 unlinked exchanges (0 types)
		


(3, 8, 0, 0)

In [10]:
if xl_importer.statistics(print_stats=False)[3] == 0:
    xl_importer.write_database()

100%|██████████| 3/3 [00:00<00:00, 1503.33it/s]

Vacuuming database 
Created database: bike_production_example


In [11]:
bike_db = bd.Database("bike_production_example")

In [12]:
for a in bike_db:
    print(f"{a} -> {a['code']}")

'natural gas production' (mega joule, NO, None) -> 6ba895871c694e9a898ade17ab6748e4
'production of bike' (unit, DK, None) -> 1d31f61bf8884a6fa5b3783270091f9d
'carbon fibre production' (kilogram, DE, None) -> 7bde0c388a1b401c95dccd0a3429bd0c


## Extract the data from the SDF file

In [13]:
from pathlib import Path

from bw_scenarios.importer import SDFImporter

file_path = Path("sdf.xlsx")

# store the scenario data in a variable
scenarios = SDFImporter.from_excel(file_path)

In the .sdf-file it is defined how the activities are changing within the distinct scenarios.

The file used for this example is specified above.

In the graphic below, the direction of change per scenario and exchange is visualized.

![scenarios_visualized.png](scenarios_visualized.png)